In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader, Dataset, ConcatDataset
from sklearn.model_selection import train_test_split
from itertools import cycle

In [2]:
# class NN(nn.Module):
#     def __init__(self, layers):
#         super(NN, self).__init__()
#         self.layers = layers
#         self.u = self._make_layers()

#     def _make_layers(self):
#         layers = []
#         for i in range(len(self.layers) - 2):
#             layers.append(nn.Linear(self.layers[i], self.layers[i+1]))
#             layers.append(nn.Tanh())
#         layers.append(nn.Linear(self.layers[-2], self.layers[-1]))
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         return self.u(x)

#     def init_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Linear):
#                 init.xavier_uniform_(m.weight)
#                 if m.bias is not None:
#                     init.zeros_(m.bias)

In [3]:
class NN(nn.Module):
    '''
    input : nodes--- list
                form [input features, hiddenlayer1, hiddenlayer2,...., outputfeatures]
    '''
    def __init__(self, nodes):
        super(NN, self).__init__()
        self.act = nn.Tanh()

        self.hidden_layers = nn.ModuleList()
        for node in range(len(nodes)-2):
            self.hidden_layers.append(nn.Linear(nodes[node], nodes[node+1]))
        self.output_layer = nn.Linear(nodes[-2] , nodes[-1])
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.zeros_(m.bias)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.act(layer(x))
        x = self.output_layer(x)
        return x

In [4]:
class PINN(nn.Module):
    def __init__(self, layers, bounds_1, bounds_2 ):
        super(PINN, self).__init__()
        
        # u: T_e(1)
        # v: T_l(1)
        self.net = NN(layers)
        self.net_2 = NN(layers)
        self.net.init_weights()
        self.net_2.init_weights()
        
        # for the first material, Au
        lb_1 = bounds_1[0]
        ub_1 = bounds_1[1]
        self.lb_1 = lb_1
        self.ub_1 = ub_1
        
        # for the second material, Cr
        lb_2 = bounds_2[0]
        ub_2 = bounds_2[1]
        self.lb_2 = lb_2
        self.ub_2 = ub_2
        
        
    def forward(self, X, t, material):
        if material==1:
            X = 2.0 * ( X - self.lb_1[0] ) / ( self.ub_1[0] - self.lb_1[0] ) - 1.0
            u = self.net(torch.cat([X, t], dim=1))
            u_x = torch.autograd.grad(u, X, torch.ones_like(u), create_graph=True)[0]
            
            v = self.net_2(torch.cat([X, t], dim=1))
            # v.requires_grad = True
            v_x = torch.autograd.grad(v, X, torch.ones_like(v), create_graph=True)[0]
            return u, u_x, v, v_x
        
        else:
            X_2 = 2.0 * ( X - self.lb_2[0] ) / ( self.ub_2[0] - self.lb_2[0] ) - 1.0
            u_2 = self.net(torch.cat([X_2, t], dim=1))
            u_x_2 = torch.autograd.grad(u_2, X_2, torch.ones_like(u_2), create_graph=True)[0]
            
            v_2 = self.net_2(torch.cat([X_2, t], dim=1))
            v_x_2 = torch.autograd.grad(v_2, X_2, torch.ones_like(v_2), create_graph=True)[0]
        return u_2, u_x_2, v_2, v_x_2
    
    def f_uv(self,x, t, material):
        if material==1:
            u, u_x, v, v_x = self.forward(x, t, material=1)

            u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0] 
            
            u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
            
            v_t = torch.autograd.grad(v, t, torch.ones_like(v), create_graph=True)[0]
            # v_xx = torch.autograd.grad(v_x, x, torch.ones_like(v_x), create_graph=True)[0]

            ce=2.1E-5*(u+1)
            cl=2.5E-3
            G=0.000026
            k=3.15E-5
            tp=0.1
            delta=0.153
            R=0.93
            J=1.666667E-2
            pow = x/delta+2.77*((t-2*tp)/(tp))**2
            sp = (0.94)*((1-R)/(tp*delta))*J
            S= (sp/ce)*torch.exp(-pow)
            k_ce = 1.50*(1/(u+1))
            
            f_u = u_t - k_ce*((u+1)/(v+1))*u_xx - (k_ce)*(u_x*(v+1)-v_x*(u+1))/((v+1)**2)*(u_x) + (G/ce)*(u-v) - S
            f_v = v_t - (G/cl)*(u-v)

            return f_u, f_v
        
        else:
            x_2 = x
            t_2 = t
            u_2, u_x_2, v_2, v_x_2 = self.forward(x_2, t_2, material=2)
            
            u_t_2 = torch.autograd.grad(u_2, t_2, torch.ones_like(u_2), create_graph=True)[0]
            u_xx_2 = torch.autograd.grad(u_x_2, x_2, torch.ones_like(u_x_2), create_graph=True)[0]
            
            v_t_2 = torch.autograd.grad(v_2, t_2, torch.ones_like(v_2), create_graph=True)[0]
            # v_xx_2 = torch.autograd.grad(v_x_2, x_2, torch.ones_like(v_x_2), create_graph=True)[0]
            
            ce_2=5.8E-5*(u_2+1)
            cl_2=3.3E-3
            G_2 = 0.00042
            k_2=9.4E-6
            tp=0.1
            delta=0.153
            R=0.93
            J=1.666667E-2
            pow = x_2/delta+2.77*((t_2-2*tp)/(tp))**2
            sp = (0.94)*((1-R)/(tp*delta))*J
            S= (sp/ce_2)*torch.exp(-pow)
            
            k_ce_2 = 1.62E-1*(1/(u_2+1))
            f_u_2 = u_t_2 - k_ce_2*((u_2+1)/(v_2+1))*u_xx_2 - (k_ce_2)*(u_x_2*(v_2+1)-v_x_2*(u_2+1))/((v_2+1)**2)*(u_x_2) + (G_2/ce_2)*(u_2-v_2) - S
            f_v_2 = v_t_2 - (G_2/cl_2)*(u_2-v_2)

            return f_u_2, f_v_2
        
        

    # def forward_1(self, X, t):
    #     X = 2.0 * ( X - self.lb ) / ( self.ub - self.lb ) - 1.0
    #     u = self.net_1(torch.cat([X, t], dim=1))
    #     u_x = torch.autograd.grad(u, X, torch.ones_like(u), create_graph=True)[0]
        
    #     v = self.net_2(torch.cat([X, t], dim=1))
    #     v_x = torch.autograd.grad(v, X, torch.ones_like(v), create_graph=True)[0]
    #     return u, u_x, v, v_x   # T_e, T_e_x, T_l, T_l_x all for material 1
    
    # For material 2, Cr
    # def forward_2(self, X_2, t_2):
    #     X_2 = 2.0 * ( X_2 - self.lb_2 ) / ( self.ub_2 - self.lb_2 ) - 1.0
    #     u_2 = self.u(torch.cat([X_2, t_2], dim=1))
    #     u_x_2 = torch.autograd.grad(u_2, X_2, torch.ones_like(u_2), create_graph=True)[0]
        
    #     v_2 = self.v(torch.cat([X_2, t_2], dim=1))
    #     v_x_2 = torch.autograd.grad(v_2, X_2, torch.ones_like(v_2), create_graph=True)[0]
    #     return u_2, u_x_2, v_2, v_x_2
    
    # For the first material (Au)
    
    ### Change the u_xx later
    # def f_uv_1(self, x, t):
        # u, u_x, v, v_x = self.forward_1(x, t)
        
        # u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0] 
        # u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
        
        # v_t = torch.autograd.grad(v, t, torch.ones_like(v), create_graph=True)[0]
        # # v_xx = torch.autograd.grad(v_x, x, torch.ones_like(v_x), create_graph=True)[0]

        # ce=2.1E-5*(u+1)
        # cl=2.5E-3
        # G=0.000026
        # k=3.15E-5
        # tp=0.1
        # delta=0.153
        # R=0.93
        # J=1.666667E-2
        # pow = x/delta+2.77*((t-2*tp)/(tp))**2
        # sp = (0.94)*((1-R)/(tp*delta))*J
        # S= (sp/ce)*torch.math.exp(-pow)
        # k_ce = 1.50*(1/(u+1))
        
        # f_u = u_t - k_ce*((u+1)/(v+1))*u_xx - (k_ce)*(u_x*(v+1)-v_x*(u+1))/((v+1)**2)*(u_x) + (G/ce)*(u-v) - S
        # f_v = v_t - (G/cl)*(u-v)

        # return f_u, f_v
    
    # def f_uv_2(self, x_2, t_2):
        # u_2, u_x_2, v_2, v_x_2 = self.forward(x_2, t_2)
        
        # u_t_2 = torch.autograd.grad(u_2, t_2, torch.ones_like(u_2), create_graph=True)[0]
        # u_xx_2 = torch.autograd.grad(u_x_2, x_2, torch.ones_like(u_x_2), create_graph=True)[0]
        
        # v_t_2 = torch.autograd.grad(v_2, t_2, torch.ones_like(v_2), create_graph=True)[0]
        # # v_xx_2 = torch.autograd.grad(v_x_2, x_2, torch.ones_like(v_x_2), create_graph=True)[0]
        
        # ce_2=5.8E-5*(u_2+1)
        # cl_2=3.3E-3
        # G_2 = 0.00042
        # k_2=9.4E-6
        # tp=0.1
        # delta=0.153
        # R=0.93
        # J=1.666667E-2
        # pow = x_2/delta+2.77*((t_2-2*tp)/(tp))**2
        # sp = (0.94)*((1-R)/(tp*delta))*J
        # S= (sp/ce_2)*torch.math.exp(-pow)
        
        # k_ce_2 = 1.62E-1*(1/(u_2+1))
        # f_u_2 = u_t_2 - k_ce_2*((u_2+1)/(v_2+1))*u_xx_2 - (k_ce_2)*(u_x_2*(v_2+1)-v_x_2*(u_2+1))/((v_2+1)**2)*(u_x_2) + (G_2/ce_2)*(u_2-v_2) - S
        # f_v_2 = v_t_2 - (G_2/cl_2)*(u_2-v_2)

        # return f_u_2, f_v_2
    

In [5]:

def loss_fn( f_u, f_v, f_u_2, f_v_2, u_x_lb, u_x_2_ub, v_x_lb, v_x_2_ub, u_1_interface, u_2_interface, v_1_interface, v_2_interface, u_x_1_interface, u_x_2_interface, u_1_initial, T_e_1_initial, v_1_initial, T_l_1_initial, u_2_initial, T_e_2_initial, v_2_initial, T_l_2_initial):
    loss = 0.0
    k1 = 3.15E-5
    k2 = 9.4E-6
    
    mse = nn.MSELoss()
    
    # loss(1) PDE1
    l1 = mse(f_u, torch.zeros_like(f_u))
    
    # loss(2) PDE1
    l2 = mse(f_v, torch.zeros_like(f_v))
    
    # loss(1) PDE2
    l3 = mse(f_u_2, torch.zeros_like(f_u_2))
    
    # loss(2) PDE2
    l4 = mse(f_v_2, torch.zeros_like(f_v_2))
    
    #loss(1) Insulated cond 1
    l5 = mse( u_x_lb, u_x_2_ub )
    
    # loss(2) Insulated cond 2
    l6 = mse( v_x_lb, v_x_2_ub )
    
    # interface loss 1
    l7 = mse( u_1_interface, u_2_interface )
    
    # interface loss 2
    vec = k1*((u_1_interface+1)/(v_1_interface+1))*u_x_2_interface - k2*((u_2_interface+1)/(v_2_interface+1))*u_x_2_interface
    
    l8 = mse( vec, torch.zeros_like(vec) ) 
    
    # initial condition(1) 1
    l9 = mse( u_1_initial, T_e_1_initial )
    
    # initial condition(1) 2
    l10 = mse( v_1_initial, T_l_1_initial )
    
    # initial condition(2) 1
    l11 = mse( u_2_initial, T_e_2_initial )
    
    # initial condition(2) 2
    l12 = mse( v_2_initial, T_l_2_initial )
    
    loss = l1 + l2 + l3 + l4 + l5 + l6 + l7 + l8 + l9 + l10 + l11 + l12
    return loss

In [6]:
def dataloader_pinn(x, test_size=0.2):
    x_train, x_test = train_test_split(x, test_size=test_size, random_state=42)
    train_dataset = DataLoader(TensorDataset(x_train.float()), batch_size=32, shuffle=True)
    test_dataset = DataLoader(TensorDataset(x_test.float()), batch_size=32, shuffle=False)
    # train_dataset = TensorDataset(x_train.float())
    # test_dataset = TensorDataset(x_test.float())

    return train_dataset, test_dataset

In [7]:
# def create_dataset():
    
#     lb = torch.Tensor([0.0, 0.0])
#     ub = torch.Tensor([0.5, 1.0])
    
#     lb2 = torch.Tensor([0.5, 0.0])
#     ub2 = torch.Tensor([1.0, 1.0])
    
#     # Au
#     x = torch.linspace(0.0,0.5,100)
#     t = torch.linspace(0.0,1.0,300)
#     X, T = torch.meshgrid(x, t)
#     X_star = torch.hstack((X.flatten()[:, None], T.flatten()[:, None]))
#     # u_star = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
#     # v_star = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
#     X_star.requires_grad = True
#     # u_star.requires_grad = True
#     # v_star.requires_grad = True
    
        
#     # Cr
#     x_2 = torch.linspace(0.5,1.0,100)
#     t_2 = torch.linspace(0.0,1.0,300)
#     X_2, T_2 = torch.meshgrid(x_2, t_2)
#     X_star_2 = torch.hstack((X_2.flatten()[:, None], T_2.flatten()[:, None]))
#     # u_star_2 = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
#     # v_star_2 = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
#     X_star_2.requires_grad = True
#     # u_star_2.requires_grad = True
#     # v_star_2.requires_grad = True
    
    
#     train_dataloader, test_dataloader = dataloader_pinn(X_star)
#     train_dataloader_2, test_dataloader_2 = dataloader_pinn(X_star_2)
    
#     # initial condition
#     X_1_initial=X_star[X_star[:,1]==0]
#     X_2_initial= X_star_2[X_star_2[:,1]==0]
    
#     # boundary condition
#     X_1_bounds_left=X_star[X_star[:,0]==lb[0]]
#     X_1_bounds_interface=X_star[X_star[:,0]==ub[0]]

    
#     X_2_bounds_interface=X_star_2[X_star_2[:,0]==lb2[0]]
#     X_2_bounds_right=X_star_2[X_star_2[:,0]==ub2[1]]
    
#     dataset_initial = DataLoader( TensorDataset(X_1_initial,X_2_initial), batch_size=32)
#     dataset_boundaries = DataLoader( TensorDataset( X_1_bounds_left, X_2_bounds_right), batch_size=32)
#     dataset_interface = DataLoader( TensorDataset( X_1_bounds_interface, X_2_bounds_interface), batch_size=32)

#     DL = ConcatDataset( ( train_dataloader, TensorDataset(X_1_initial,X_2_initial), TensorDataset( X_1_bounds_left, X_2_bounds_right), TensorDataset( X_1_bounds_interface, X_2_bounds_interface)  ) )
    
#     # loader_IC = DataLoader(train_dataset_initial, batch_size=32, shuffle=True)
#     # loader_BC = DataLoader(train_dataset_boundaries, batch_size=32, shuffle=True)
    
    
#     return (train_dataloader, train_dataloader_2), (test_dataloader, test_dataloader_2), dataset_initial, dataset_boundaries, dataset_interface, DL

# tuple_train_dataloader, tuple_test_dataloader, dataloader_initial, dataloader_boundaries, dataloader_interface, DL = create_dataset()
# zipped_train_data = zip(cycle(tuple_train_dataloader), dataloader_initial, dataloader_boundaries, dataloader_interface)

In [8]:
def create_dataset():
    
    lb = torch.Tensor([0.0, 0.0])
    ub = torch.Tensor([0.5, 1.0])
    
    lb2 = torch.Tensor([0.5, 0.0])
    ub2 = torch.Tensor([1.0, 1.0])
    
    # Au
    x = torch.linspace(0.0,0.5,100)
    t = torch.linspace(0.0,1.0,300)
    X, T = torch.meshgrid(x, t)
    X_star = torch.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    # u_star = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
    # v_star = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
    X_star.requires_grad = True
    # u_star.requires_grad = True
    # v_star.requires_grad = True
    
        
    # Cr
    x_2 = torch.linspace(0.5,1.0,100)
    t_2 = torch.linspace(0.0,1.0,300)
    X_2, T_2 = torch.meshgrid(x_2, t_2)
    X_star_2 = torch.hstack((X_2.flatten()[:, None], T_2.flatten()[:, None]))
    # u_star_2 = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
    # v_star_2 = torch.linspace(0.0,0.0,100).flatten()[:,None].T.flatten()[:,None]
    X_star_2.requires_grad = True
    # u_star_2.requires_grad = True
    # v_star_2.requires_grad = True
    
    
    train_dataloader, test_dataloader = dataloader_pinn(X_star)
    train_dataloader_2, test_dataloader_2 = dataloader_pinn(X_star_2)
    
    # initial condition
    X_1_initial=X_star[X_star[:,1]==0]
    X_2_initial= X_star_2[X_star_2[:,1]==0]
    
    # boundary condition
    X_1_bounds_left=X_star[X_star[:,0]==lb[0]]
    X_1_bounds_interface=X_star[X_star[:,0]==ub[0]]

    
    X_2_bounds_interface=X_star_2[X_star_2[:,0]==lb2[0]]
    X_2_bounds_right=X_star_2[X_star_2[:,0]==ub2[1]]
    
    dataset_initial = DataLoader( TensorDataset(X_1_initial,X_2_initial), batch_size=32)
    dataset_boundaries = DataLoader( TensorDataset( X_1_bounds_left, X_2_bounds_right), batch_size=32)
    dataset_interface = DataLoader( TensorDataset( X_1_bounds_interface, X_2_bounds_interface), batch_size=32)

    return (train_dataloader, train_dataloader_2), (test_dataloader, test_dataloader_2), TensorDataset(X_1_initial,X_2_initial), TensorDataset( X_1_bounds_left, X_2_bounds_right), TensorDataset( X_1_bounds_interface, X_2_bounds_interface)

tuple_train_dataloader, tuple_test_dataloader, ds_initial, ds_boundaries, ds_interface = create_dataset()
# zipped_train_data = zip(cycle(tuple_train_dataloader), ds_initial, ds_boundaries, ds_interface)

/opt/anaconda3/envs/deeponet/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
def train_pinn(pinn: PINN, epochs):
    train_losses = []
    network = pinn.net
    network_2 = pinn.net_2
    network.train()
    network_2.train()
    
    optimizer = torch.optim.Adam( list(network.parameters()) + list(network_2.parameters()), lr=0.0001 )
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        # for tr_loader, ds_initial, ds_boundaries, ds_interface in DL:
        for tr_loader_1, tr_loader_2, ds_initial_, ds_boundaries_, ds_interface_  in zip(tuple_train_dataloader[0], tuple_train_dataloader[1], DataLoader(ds_initial, batch_size=32), DataLoader(ds_boundaries, batch_size=32), DataLoader(ds_interface, batch_size=32)):
        # for batch_idx, (tr_loader, ds_initial, ds_boundaries, ds_interface) in enumerate(zipped_train_data):
            # tr_loader_1, tr_loader_2 = tr_loader
            
            
            
            # tr_loader_1, tr_loader_2 = zip_train
            X_1_initial, X_2_initial = ds_initial_
            X_1_bounds_interface, X_2_bounds_interface = ds_interface_
            X_1_bounds_left, X_2_bounds_right = ds_boundaries_
            
            X_star = tr_loader_1[0]
            X_star_2 = tr_loader_2[0]
            
            # full grid
            ## X_star: [X_full, T_full]; X_full \belongsto [0,0.5], T_full \belongsto [0,1]
            X_f = X_star[:, 0:1]
            t_f = X_star[:, 1:2]
            
            ## X_star_2: [X_full, T_full]; X_full \belongsto [0.5,1], T_full \belongsto [0,1]
            X_f_2 = X_star_2[:, 0:1]
            t_f_2 = X_star_2[:, 1:2]
            
            # interface- left boundary
            X_int_1 = X_1_bounds_interface[:,0:1]
            t_int_1 = X_1_bounds_interface[:,1:2]
            
            X_int_2 = X_2_bounds_interface[:,0:1]
            t_int_2 = X_2_bounds_interface[:,1:2]
            
            # left boundary
            X_lb_1 = X_1_bounds_left[:,0:1]
            t_lb_1 = X_1_bounds_left[:,1:2]
            
            
            # upper/right boundary
            X_ub_2 = X_2_bounds_right[:,0:1]
            t_ub_2 = X_2_bounds_right[:,1:2]
            
            # initial condition
            X_initial_1 = X_1_initial[:, 0:1]
            t_initial_1 = X_1_initial[:, 1:2]
            
            X_initial_2 = X_2_initial[:, 0:1]
            t_initial_2 = X_2_initial[:, 1:2]

            
            optimizer.zero_grad()

            # Over the domain   
            # f_u, f_v = pinn.f_uv_1(X_f, t_f)
            # f_u_2, f_v_2 = pinn.f_uv_2(X_f_2, t_f_2)
            f_u, f_v = pinn.f_uv(X_f, t_f, material=1)
            f_u_2, f_v_2 = pinn.f_uv(X_f_2, t_f_2, material=2)
            
            # left boundary conditions - 1
            # _, u_x_lb, _, v_x_lb = pinn.forward_1(X_lb_1, t_lb_1)
            _, u_x_lb, _, v_x_lb = pinn.forward(X_lb_1, t_lb_1, material=1)
            
            # right/upper boundary conditions - 2
            # _, u_x_ub_2, _, v_x_ub_2 = pinn.forward_2(X_ub_2, t_ub_2)
            _, u_x_ub_2, _, v_x_ub_2 = pinn.forward(X_ub_2, t_ub_2, material=2)
            
            # interface condition(1)
            # u_interface_1, u_x_interface_1, v_interface_1, _ = pinn.forward_1( X_int_1, t_int_1 )
            # u_interface_2, u_x_interface_2, v_interface_2, _ = pinn.forward_2( X_int_2, t_int_2 )
            u_interface_1, u_x_interface_1, v_interface_1, _ = pinn.forward( X_int_1, t_int_1, material=1 )
            u_interface_2, u_x_interface_2, v_interface_2, _ = pinn.forward( X_int_2, t_int_2, material=2)
            
            # initial conditions(1)
            # u_initial_1, _, v_initial_1, _ = pinn.forward_1( X_initial_1, t_initial_1 )
            # u_initial_2, _, v_initial_2, _ = pinn.forward_2( X_initial_2, t_initial_2 )
            u_initial_1, _, v_initial_1, _ = pinn.forward( X_initial_1, t_initial_1, material=1 )
            u_initial_2, _, v_initial_2, _ = pinn.forward( X_initial_2, t_initial_2, material=2 )
            
            ## setting up the initial condition
            T_e_initial_1 = torch.zeros_like(u_initial_1)
            T_e_initial_2 = torch.zeros_like(u_initial_2)
            T_l_initial_1 = torch.zeros_like(v_initial_1)
            T_l_initial_2 = torch.zeros_like(v_initial_2)
            
            loss = loss_fn( f_u, f_v, f_u_2, f_v_2, u_x_lb, u_x_ub_2, v_x_lb, v_x_ub_2, u_interface_1, u_interface_2, v_interface_1, v_interface_2, u_x_interface_1, u_x_interface_2, u_initial_1, T_e_initial_1, v_initial_1, T_l_initial_1, u_initial_2, T_e_initial_2, v_initial_2, T_l_initial_2)

            loss.backward(retain_graph=True)
            optimizer.step()
            epoch_loss = epoch_loss + loss.item()
            
        if epoch % 100 == 0:
            print(f'Epoch {epoch} Loss: {epoch_loss}')
        train_losses.append(epoch_loss / len(tr_loader_1))
        
    return train_losses
            
    

In [10]:
layers = [2, 80, 80, 80, 80, 80,  1]
bounds_1 = torch.Tensor( [ [0.0, 0.0], [0.5, 1.0] ] )
bounds_2 = torch.Tensor( [ [0.5, 0.0], [1.0, 1.0] ] )
pinn = PINN(layers, bounds_1 , bounds_2 )

In [14]:
train_losses = train_pinn(pinn, epochs=1000)

Epoch 0 Loss: 33373.92022705078
Epoch 100 Loss: 13113.009521484375
Epoch 200 Loss: 23704.0390625
Epoch 300 Loss: 9666.203109741211
Epoch 400 Loss: 14339.057861328125
Epoch 500 Loss: 7318.112854003906
Epoch 600 Loss: 15853.283752441406
Epoch 700 Loss: 5268.3204345703125
Epoch 800 Loss: 8387.764038085938
Epoch 900 Loss: 6256.1632080078125


In [13]:
def test_pinn(pinn: PINN, optimizer, layers, tuple_test_dataloader):
    
    test_losses = []
    network = pinn.net(layers)
    network_2 = pinn.net_2(layers)
    network.eval()
    network_2.eval()

    test_loss = 0.0
    all_outputs = []
    with torch.no_grad():
        for test_dl_1, test_dl_2 in tuple_test_dataloader:
            
            X_star = test_dl_1
            X_star_2 = test_dl_2
            
            X = X_star[:,0:1]
            t = X_star[:,1:2]
            
            X_2 = X_star_2[:,0:1]
            t_2 = X_star_2[:, 1:2]
            
            u_1, u_x_1, v_1, v_x_1 = pinn.forward(X,t, material=1)
            u_2, u_x_2, v_2, v_x_2 = pinn.forward(X_2, t_2, material=2)

            ### TEST LOSS???
            loss = optimizer(outputs, targets)
            test_loss += loss.item()
            all_outputs.append(outputs)
    test_loss /= len(test_dl_1)
    # print('Test Loss: %.6f' % test_loss)

    return torch.cat(all_outputs), test_loss